In [4]:
# !pip3 install xgboost numpy textblob keras
from time import time
import pandas as pd
import numpy as np
import re
import csv
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn import decomposition, ensemble
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
# import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [5]:
df = pd.read_csv('tweets_very_clean1.csv')

In [6]:
vol = pd.read_csv('vol.csv')

In [7]:
df.merge(vol, on='date', how='left')

,Unnamed: 0,index,date,id,user,fullname,url,timestamp,replies,likes,...,text,lang,year,month,day,text1,tags,promote,clean,vol
0,0,0,2017-12-05,9.379548e+17,BisqArviraj,Ar Viraj,,2017-12-05 07:00:16+00,0.0,0.0,...,SELL\nPayment method: SEPA\nOffer ID: 90196354...,en,2017,12,5,SELL\nPayment method: SEPA\nOffer ID: 90196354...,,,sell payment method sepa offer id amount btc p...,BULL
1,1,1,2017-12-05,9.379497e+17,cryptolligence,Cryptolligence,,2017-12-05 06:40:05+00,0.0,0.0,...,Buy! (11:40:05 pm PDT)\nPrice: 11720.00 (+/-...,en,2017,12,5,Buy! (11:40:05 pm PDT)\nPrice: 11720.00 (+/-...,coinbase gdax btc bitcoin trading,,buy pm pdt price close stop gdax coinbase btc ...,BULL
2,2,2,2017-12-05,9.379497e+17,cryptolligence,Cryptolligence,,2017-12-05 06:40:10+00,0.0,0.0,...,Buy! (11:40:10 pm PDT)\nPrice: 11721.00 (+/-...,en,2017,12,5,Buy! (11:40:10 pm PDT)\nPrice: 11721.00 (+/-...,coinbase gdax btc bitcoin trading,,buy pm pdt price close stop gdax coinbase btc ...,BULL
3,3,3,2017-12-05,9.379547e+17,coinstats,Coinstats,,2017-12-05 07:00:01+00,0.0,0.0,...,"Bitcoin - BTC\nPrice: $11,861.80\nChange in 1h...",en,2017,12,5,"Bitcoin - BTC\nPrice: $11,861.80\nChange in 1h...",Bitcoin BTC,,bitcoin btc price change h market cap ranking ...,BULL
4,4,4,2017-12-05,9.379536e+17,TNFXtrading,TNFX Trading,http://t.me/TNFXtrading,2017-12-05 06:55:38+00,0.0,0.0,...,تنحصر تداولات سعر بتكوين الأخيرة داخل نموذج عل...,en,2017,12,5,تنحصر تداولات سعر بتكوين الأخيرة داخل نموذج عل...,USDpic.twitter.com/oh8SKBx4DE BTC BTCUSD,,تنحصر تداولات سعر بتكوين الأخيرة داخل نموذج عل...,BULL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332456,332456,1324380,2019-05-14,1.128253e+18,cointrend_jp,COINTREND,https://cointrend.jp/,2019-05-14 08:58:01+00,0.0,0.0,...,ツイート数の多かった仮想通貨\n1位 $BTC 650 Tweets\n2位 $TRX 26...,en,2019,5,14,ツイート数の多かった仮想通貨\n1位 $BTC 650 Tweets\n2位 $TRX 26...,,,ツイート数の多かった仮想通貨 位 btc tweets 位 trx tweets 位 xrp...,BULL
332457,332457,1324381,2019-05-14,1.128249e+18,SssaPTAMUo5R7RE,Александр Свечкар,https://bi.city/s/KU0Gf,2019-05-14 08:41:43+00,0.0,0.0,...,[BTC Dips Below 8000 USDT]\n\nCiting data from...,en,2019,5,14,[BTC Dips Below 8000 USDT]\n\nCiting data from...,,,btc dips usdt citing data binance price bitcoi...,BULL
332458,332458,1324382,2019-05-14,1.128251e+18,tr_ticker,Tr Ticker Bot,,2019-05-14 08:50:40+00,0.0,0.0,...,#Doviz\n-------------------\n#USD : 6.1077\n#E...,en,2019,5,14,#Doviz\n-------------------\n#USD : 6.1077\n#E...,Doviz BtcTurk BTC EUR Koinim Paribu GBP Gobaba...,,doviz usd eur gbp btc gobaba btcturk koinim pa...,BULL
332459,332459,1324383,2019-05-14,1.128249e+18,EconomiesCom,Economies.com,http://dlvr.it/R4g63l,2019-05-14 08:41:02+00,0.0,0.0,...,Midday update for bitcoin 14-05-2019: Bitcoin ...,en,2019,5,14,Midday update for bitcoin 14-05-2019: Bitcoin ...,,,midday update bitcoin bitcoin price resumes po...,BULL


In [8]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/user/nltk_data...


True

In [9]:
def vader_compound(text):
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(text)
    compound, positive, negative, neutral = ss["compound"], ss["pos"], ss["neg"], ss["neu"]
    if compound > 0:
      return "BULL"
    return "BEAR"

In [11]:
df['sent'] = df['clean'].apply(lambda x: vader_compound(str(x)))

In [15]:
df = df.merge(vol, on='date', how='left')
df.tail(3)

,Unnamed: 0,index,date,id,user,fullname,url,timestamp,replies,likes,...,lang,year,month,day,text1,tags,promote,clean,sent,vol
332458,332458,1324382,2019-05-14,1.128251e+18,tr_ticker,Tr Ticker Bot,,2019-05-14 08:50:40+00,0.0,0.0,...,en,2019,5,14,#Doviz\n-------------------\n#USD : 6.1077\n#E...,Doviz BtcTurk BTC EUR Koinim Paribu GBP Gobaba...,,doviz usd eur gbp btc gobaba btcturk koinim pa...,BEAR,BULL
332459,332459,1324383,2019-05-14,1.128249e+18,EconomiesCom,Economies.com,http://dlvr.it/R4g63l,2019-05-14 08:41:02+00,0.0,0.0,...,en,2019,5,14,Midday update for bitcoin 14-05-2019: Bitcoin ...,,,midday update bitcoin bitcoin price resumes po...,BULL,BULL
332460,332460,1324384,2019-05-22,1.131211e+18,sylviaNargaard1,sylvia Nargaard,,2019-05-22 12:51:23+00,0.0,5.0,...,en,2019,5,22,"Happy pizza Day!\nBack in 2010, Hanyecz bought...",btc bitcoin,,happy pizza day back hanyecz bought two papa j...,BULL,BEAR


In [16]:
correct = 0
count = 0  
for o, c in zip(df['sent'].iteritems(), df['vol'].iteritems()):
  index = o[0]
  op = o[1]
  cl = c[1]
  if op == cl:
    correct += 1
  count += 1
accuracy = float(correct) / float(count)
print(accuracy)

0.5116148961833117


In [ ]:
df = df.dropna(subset=['clean'])

In [ ]:
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=500, stop_words='english',analyzer='word', token_pattern=r'\w{1,}')
bow = bow_vectorizer.fit_transform(df['clean'])
bow.shape

In [ ]:
ditc= bow_vectorizer.vocabulary_
print(ditc)

In [ ]:
print(bow_vectorizer.get_feature_names())

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=500, stop_words='english',analyzer='word', token_pattern=r'\w{1,}')
tfidf = tfidf_vectorizer.fit_transform(df['clean'])
tfidf.shape

In [ ]:
ditc2 = tfidf_vectorizer.vocabulary_
print(ditc2)

In [ ]:
print(tfidf_vectorizer.get_feature_names())

In [ ]:
vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=500)
ngram = vect_ngram.fit_transform(df['clean'])
ngram.shape

In [ ]:
#2- CREATING a FAKE Y
#ate 11 dec 2017
#ate 10 dec 2018
#ate end

def senti(x):
  if x < 2018:
    return 'BULL'
  elif x > 2018:
    return 'BULL'
  else:
    return 'BEAR'

df['sent'] = df['year'].apply(lambda x: senti(x) )
df.tail(3)

In [ ]:
df['sent'].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df['sent'])

In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return (metrics.accuracy_score(predictions, y_test), metrics.precision_score(predictions, y_test))

In [ ]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), X_count_train, y_train, X_count_test)
print("NB, on Bag of Words : Accuracy, Precision ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), X_tfidf_train, y_train, X_tfidf_test)
print("NB, on TF-IDF  : Accuracy, Precision ", accuracy)

In [ ]:
# Log Reg Classifier on Count Vectors - takes 4 min
accuracy = train_model(linear_model.LogisticRegression(), X_count_train, y_train, X_count_test)
print("LR, Bag of Words: : Accuracy, Precision ", accuracy)

# Log Reg Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(solver='lbfgs', max_iter=100), X_tfidf_train, y_train, X_tfidf_test)
print("LR, on TF-IDF  : Accuracy, Precision ", accuracy)

In [ ]:
# SVM on Ngram Level TF IDF Vectors - takes forever to calculate MORE THAN 10 MN
accuracy = train_model(svm.SVC(kernel='linear'), X_ngram_train, y_train, X_ngram_test)
print("SVM,  on NGRAM  : Accuracy, Precision, Recall ", accuracy)

In [ ]:
# RF on Count Vectors - takes more than 20 min, never finished
accuracy = train_model(ensemble.RandomForestClassifier(n_estimators = 100),X_count_train, y_train, X_count_test)
print("RF, on Bag of Words : Accuracy, Precision, Recall ", accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), X_tfidf_train, y_train, X_tfidf_test)
print("RF, on TF-IDF  : Accuracy, Precision, Recall ", accuracy)

In [ ]:
# Extereme Gradient Boosting on Count Vectors - 31 mn for bag of words results and counting
accuracy = train_model(xgboost.XGBClassifier(), X_count_train.tocsc(), y_train, X_count_test.tocsc())
print("Xgb, on Bag of Words : Accuracy, Precision, Recall ", accuracy)

# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), X_tfidf_train.tocsc(), y_train, X_tfidf_test.tocsc())
print("Xgb, on TF-IDF  : Accuracy, Precision, Recall ", accuracy)

In [ ]:
#ON THE MODEL THAT SHOWS MOST ACCURACY, DIG ON ITS SPECS
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_predict_svm))
# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_predict_svm))
# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_predict_svm))